In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

# Path (same as your 4.7 notebook)
path = r'/Users/josephadamski/Instacart Basket Analysis'

# Import the ords_prods_merge dataframe
ords_prods_merge = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ords_prods_merge.pkl'))

# Check the dataframe
print(f"Shape: {ords_prods_merge.shape}")
ords_prods_merge.head()

Shape: (32404859, 19)


,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order_flag,add_to_cart_order,reordered,_merge,price_label,busiest_day,busiest_days,busiest_period_of_day
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,False,5,0,both,Mid-range product,Regularly busy,Regularly busy,Most orders
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,False,1,1,both,Mid-range product,Regularly busy,Regularly busy,Average orders
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,False,20,0,both,Mid-range product,Busiest day,Busiest days,Average orders
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,True,10,0,both,Mid-range product,Regularly busy,Slowest days,Most orders
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,False,11,1,both,Mid-range product,Least busy,Slowest days,Average orders


## 2. Department Aggregation Analysis
Calculate the mean order_number grouped by department_id for the entire dataframe.

In [2]:
# Group by department_id and calculate mean order_number for ENTIRE dataframe
dept_order_mean = ords_prods_merge.groupby('department_id')['order_number'].mean()

print("Mean order number by department:")
print(dept_order_mean)

Mean order number by department:
department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64


## Analysis: Department Order Number Means

After calculating the mean order_number grouped by department_id for the entire dataframe:

**Key Observations:**
- The full dataset (32M+ rows) provides more reliable statistics than the 1M row subset used in the exercise
- Departments with higher mean order numbers indicate products frequently purchased by loyal customers
- This aggregation shows which departments attract repeat customers vs new customers

## 4. Loyalty Flag Creation
Create a loyalty flag based on maximum orders per user using transform() and loc().

**Criteria:**
- Loyal customer: max_order > 40
- Regular customer: 10 < max_order ≤ 40
- New customer: max_order ≤ 10

In [4]:
# Step 1: Create max_order column using transform()
ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform('max')

# Step 2: Create loyalty_flag using loc()
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & 
                     (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

# Check frequency
print("Loyalty Flag Distribution:")
print(ords_prods_merge['loyalty_flag'].value_counts(dropna=False))

# Verify accuracy - check several users
print("\nSample verification:")
ords_prods_merge[['user_id', 'order_number', 'max_order', 'loyalty_flag']].head(30)

Loyalty Flag Distribution:
loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

Sample verification:


,user_id,order_number,max_order,loyalty_flag
0,138,28,32,Regular customer
1,138,30,32,Regular customer
2,709,2,5,New customer
3,764,1,3,New customer
4,764,3,3,New customer
5,777,16,26,Regular customer
6,825,3,9,New customer
7,910,12,12,Regular customer
8,1052,10,20,Regular customer
9,1052,15,20,Regular customer


## 5. Spending Habits by Loyalty Category
Analyze whether loyal customers spend differently than regular or new customers.

In [9]:
# Get detailed price statistics for each loyalty category
loyalty_price_stats = ords_prods_merge.groupby('loyalty_flag')['prices'].describe()

print("Price statistics by loyalty category:")
print(loyalty_price_stats)

# Calculate and display mean prices
print("\n" + "="*50)
print("Mean prices by customer type:")
print("="*50)
for category in ['Loyal customer', 'Regular customer', 'New customer']:
    mean_price = ords_prods_merge[ords_prods_merge['loyalty_flag'] == category]['prices'].mean()
    median_price = ords_prods_merge[ords_prods_merge['loyalty_flag'] == category]['prices'].median()
    print(f"{category}: Mean = ${mean_price:.2f}, Median = ${median_price:.2f}")

Price statistics by loyalty category:
                       count       mean         std  min  25%  50%   75%  \
loyalty_flag                                                               
Loyal customer    10284093.0  10.386336  328.017787  1.0  4.2  7.4  11.2   
New customer       6243990.0  13.294670  597.560299  1.0  4.2  7.4  11.3   
Regular customer  15876776.0  12.495717  539.720919  1.0  4.2  7.4  11.3   

                      max  
loyalty_flag               
Loyal customer    99999.0  
New customer      99999.0  
Regular customer  99999.0  

Mean prices by customer type:
Loyal customer: Mean = $10.39, Median = $7.40
Regular customer: Mean = $12.50, Median = $7.40
New customer: Mean = $13.29, Median = $7.40


### Spending Habits Analysis

After analyzing product prices across the three customer loyalty categories:

**Findings:**
- Loyal customers: Mean = $10.39, Median = $7.40
- Regular customers: Mean = $12.50, Median = $7.40
- New customers: Mean = $13.29, Median = $7.40

**Insights:**
Loyal customers spend $2.90 less per product on average compared to new customers, indicating they focus on value and everyday essentials rather than premium items. The consistent median price of $7.40 across all segments suggests typical purchases are similar, but new customers may be experimenting with higher-priced products. Marketing should emphasize value bundles and loyalty rewards for repeat customers while using premium product recommendations to attract new shoppers.

## 6. Spending Flag Creation
Create a spending flag based on average price across all user orders.

**Criteria:**
- Low spender: avg_price < $10
- High spender: avg_price ≥ $10

In [5]:
# Step 1: Calculate average price per user
ords_prods_merge['avg_price'] = ords_prods_merge.groupby('user_id')['prices'].transform('mean')

# Step 2: Create spending_flag using loc()
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low spender'

ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High spender'

# Check frequency
print("Spending Flag Distribution:")
print(ords_prods_merge['spending_flag'].value_counts(dropna=False))

# Verify with sample data
print("\nSample verification:")
ords_prods_merge[['user_id', 'prices', 'avg_price', 'spending_flag']].head(20)

Spending Flag Distribution:
spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

Sample verification:


,user_id,prices,avg_price,spending_flag
0,138,5.8,6.935811,Low spender
1,138,5.8,6.935811,Low spender
2,709,5.8,7.930208,Low spender
3,764,5.8,4.972414,Low spender
4,764,5.8,4.972414,Low spender
5,777,5.8,6.935398,Low spender
6,825,5.8,5.957576,Low spender
7,910,5.8,6.680000,Low spender
8,1052,5.8,7.162500,Low spender
9,1052,5.8,7.162500,Low spender


## 7. Order Frequency Flag Creation
Create an order frequency flag based on median days since prior order.

**Criteria:**
- Non-frequent customer: median > 20 days
- Regular customer: 10 < median ≤ 20 days
- Frequent customer: median ≤ 10 days

In [6]:
# Step 1: Calculate median days_since_prior_order per user
ords_prods_merge['median_days_prior'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform('median')

# Step 2: Create order_frequency_flag using loc()
ords_prods_merge.loc[ords_prods_merge['median_days_prior'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'

ords_prods_merge.loc[(ords_prods_merge['median_days_prior'] > 10) & 
                     (ords_prods_merge['median_days_prior'] <= 20), 'order_frequency_flag'] = 'Regular customer'

ords_prods_merge.loc[ords_prods_merge['median_days_prior'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

# Check frequency
print("Order Frequency Flag Distribution:")
print(ords_prods_merge['order_frequency_flag'].value_counts(dropna=False))

# Verify - check multiple rows per user
print("\nSample verification:")
ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_prior', 'order_frequency_flag']].head(30)

Order Frequency Flag Distribution:
order_frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64

Sample verification:


,user_id,days_since_prior_order,median_days_prior,order_frequency_flag
0,138,3.0,8.0,Frequent customer
1,138,20.0,8.0,Frequent customer
2,709,6.0,8.0,Frequent customer
3,764,NaN,9.0,Frequent customer
4,764,9.0,9.0,Frequent customer
5,777,26.0,11.0,Regular customer
6,825,30.0,20.0,Regular customer
7,910,30.0,6.0,Frequent customer
8,1052,19.0,10.0,Frequent customer
9,1052,15.0,10.0,Frequent customer


## 8. Summary of New Columns Created

This analysis added 6 new columns to the dataframe:
1. **max_order**: Maximum order number per user
2. **loyalty_flag**: Customer loyalty category (Loyal/Regular/New)
3. **avg_price**: Average price of products per user
4. **spending_flag**: Spending category (Low/High spender)
5. **median_days_prior**: Median days between orders per user
6. **order_frequency_flag**: Order frequency category (Frequent/Regular/Non-frequent)

In [7]:
# Verify all new columns exist
print("Final dataframe info:")
print(f"Shape: {ords_prods_merge.shape}")
print(f"\nNew columns added:")
new_columns = ['max_order', 'loyalty_flag', 'avg_price', 'spending_flag', 
               'median_days_prior', 'order_frequency_flag']
for col in new_columns:
    if col in ords_prods_merge.columns:
        print(f"✓ {col}")
    else:
        print(f"✗ {col} - MISSING!")

# Display sample of all new columns
ords_prods_merge[['user_id'] + new_columns].head(10)

Final dataframe info:
Shape: (32404859, 25)

New columns added:
✓ max_order
✓ loyalty_flag
✓ avg_price
✓ spending_flag
✓ median_days_prior
✓ order_frequency_flag


,user_id,max_order,loyalty_flag,avg_price,spending_flag,median_days_prior,order_frequency_flag
0,138,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
1,138,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
2,709,5,New customer,7.930208,Low spender,8.0,Frequent customer
3,764,3,New customer,4.972414,Low spender,9.0,Frequent customer
4,764,3,New customer,4.972414,Low spender,9.0,Frequent customer
5,777,26,Regular customer,6.935398,Low spender,11.0,Regular customer
6,825,9,New customer,5.957576,Low spender,20.0,Regular customer
7,910,12,Regular customer,6.680000,Low spender,6.0,Frequent customer
8,1052,20,Regular customer,7.162500,Low spender,10.0,Frequent customer
9,1052,20,Regular customer,7.162500,Low spender,10.0,Frequent customer


## 9. Export Updated Dataframe
Export the enriched dataframe back to the Prepared Data folder.

In [8]:
# Export the updated dataframe as pickle
ords_prods_merge.to_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ords_prods_merge.pkl'))

print("✓ Dataframe successfully exported to Prepared Data folder!")
print(f"✓ Final shape: {ords_prods_merge.shape}")
print(f"✓ File location: {os.path.join(path, 'Data', 'Prepared Data', 'ords_prods_merge.pkl')}")

✓ Dataframe successfully exported to Prepared Data folder!
✓ Final shape: (32404859, 25)
✓ File location: /Users/josephadamski/Instacart Basket Analysis/Data/Prepared Data/ords_prods_merge.pkl
